In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn5u" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:07<00:00,  5.18s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  54%|█████▍    | 14/26 [00:00<00:00, 130.38batch/s, loss=0.25] 

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9160 | AUPR: 0.6547 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9206 | AUPR: 0.6636 | Acc: 95.39%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9224 | AUPR: 0.6651 | Acc: 95.21%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9160 | AUPR: 0.6540 | Acc: 95.38%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9211 | AUPR: 0.6633 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9202 | AUPR: 0.6638 | Acc: 95.33%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9224 | AUPR: 0.6651 | Acc: 95.21%
Internal test: AUC: 0.9266 | AUPR: 0.6870 | Acc: 95.04%
External test: AUC: 0.9215 | AUPR: 0.6626 | Acc: 95.01%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC


model(kernel=linear, C=0.01)
Valid:         AUC: 0.9163 | AUPR: 0.6258 | Acc: 95.02%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9166 | AUPR: 0.6236 | Acc: 94.99%
model(kernel=linear, C=1)
Valid:         AUC: 0.9163 | AUPR: 0.6223 | Acc: 94.98%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9166 | AUPR: 0.6236 | Acc: 94.99%
Internal test: AUC: 0.9218 | AUPR: 0.6688 | Acc: 94.91%
External test: AUC: 0.9142 | AUPR: 0.6135 | Acc: 94.67%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7535 | AUPR: 0.4173 | Acc: 94.63%
model(n_neighbors=5)
Valid:         AUC: 0.7835 | AUPR: 0.4652 | Acc: 94.96%
model(n_neighbors=7)
Valid:         AUC: 0.8004 | AUPR: 0.4960 | Acc: 95.01%
best model(n_neighbors=7)
Valid:         AUC: 0.8004 | AUPR: 0.4960 | Acc: 95.01%
Internal test: AUC: 0.7948 | AUPR: 0.5267 | Acc: 95.01%
External test: AUC: 0.8080 | AUPR: 0.5036 | Acc: 94.81%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8744 | AUPR: 0.4473 | Acc: 83.51%
best model()
Valid:         AUC: 0.8744 | AUPR: 0.4473 | Acc: 83.51%
Internal test: AUC: 0.8822 | AUPR: 0.4791 | Acc: 84.23%
External test: AUC: 0.8712 | AUPR: 0.4244 | Acc: 82.38%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6507 | Acc: 95.22%
model(C=10.0)
Valid:         AUC: 0.9180 | AUPR: 0.6498 | Acc: 95.21%
model(C=100.0)
Valid:         AUC: 0.9180 | AUPR: 0.6493 | Acc: 95.22%
best model(C=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6507 | Acc: 95.22%
Internal test: AUC: 0.9249 | AUPR: 0.6807 | Acc: 95.13%
External test: AUC: 0.9130 | AUPR: 0.6287 | Acc: 94.91%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier


model(criterion=gini)
Valid:         AUC: 0.8667 | AUPR: 0.5561 | Acc: 82.04%
model(criterion=log_loss)
Valid:         AUC: 0.8690 | AUPR: 0.5598 | Acc: 80.31%
model(criterion=entropy)
Valid:         AUC: 0.8690 | AUPR: 0.5598 | Acc: 80.31%
best model(criterion=log_loss)
Valid:         AUC: 0.8690 | AUPR: 0.5598 | Acc: 80.31%
Internal test: AUC: 0.8808 | AUPR: 0.6172 | Acc: 79.15%
External test: AUC: 0.8763 | AUPR: 0.5849 | Acc: 77.84%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9096 | AUPR: 0.6451 | Acc: 88.75%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9097 | AUPR: 0.6449 | Acc: 88.89%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9101 | AUPR: 0.6462 | Acc: 88.96%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9107 | AUPR: 0.6484 | Acc: 89.27%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9112 | AUPR: 0.6500 | Acc: 89.34%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9112 | AUPR: 0.6497 | Acc: 89.36%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9107 | AUPR: 0.6484 | Acc: 89.27%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9112 | AUPR: 0.6500 | Acc: 89.34%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9112 | AUPR: 0.6497 | Acc: 89.36%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9112 | AUPR: 0.6497 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9165 | AUPR: 0.6473 | Acc: 95.19%
model(criterion=squared_error)
Valid:         AUC: 0.9168 | AUPR: 0.6465 | Acc: 95.20%
best model(criterion=squared_error)
Valid:         AUC: 0.9168 | AUPR: 0.6465 | Acc: 95.20%
Internal test: AUC: 0.9263 | AUPR: 0.6917 | Acc: 95.09%
External test: AUC: 0.9214 | AUPR: 0.6656 | Acc: 95.18%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9135 | AUPR: 0.6231 | Acc: 95.02%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9137 | AUPR: 0.6243 | Acc: 95.03%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9134 | AUPR: 0.6248 | Acc: 95.08%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9137 | AUPR: 0.6243 | Acc: 95.03%
Internal test: AUC: 0.9184 | AUPR: 0.6711 | Acc: 95.11%
External test: AUC: 0.9137 | AUPR: 0.6162 | Acc: 94.91%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)